In [1]:
from pathlib import Path
import numpy as np
from queue import PriorityQueue

In [2]:
fp = "data/18.txt"
coords = [
    (int(x.split(",")[1]), int(x.split(",")[0]))
    for x in Path(fp).read_text().strip().split("\n")
]

In [3]:
size, n = (6, 12) if "sample" in fp else (70, 1024)
size += 1
grid = np.zeros((size, size))

for x, y in coords[:n]:
    grid[x, y] = 1

In [4]:
def heuristic(x, y, goal_x, goal_y):
    return np.abs(x - goal_x) + np.abs(y - goal_y)


def get_neighbors(x, y):
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        if 0 <= x + dx < grid.shape[0] and 0 <= y + dy < grid.shape[1]:
            if grid[x + dx, y + dy] != 1:
                yield x + dx, y + dy

In [5]:
def cost(grid):
    start = (0, 0)
    goal = (size - 1, size - 1)

    frontier = PriorityQueue()
    frontier.put((0, start))

    cost_so_far = {start_state: 0 for _, start_state in frontier.queue}

    while not frontier.empty():
        _, current = frontier.get()
        if current == goal:
            break

        for next_state in get_neighbors(*current):
            new_cost = cost_so_far[current] + 1
            if next_state not in cost_so_far or new_cost < cost_so_far[next_state]:
                cost_so_far[next_state] = new_cost
                priority = new_cost + heuristic(*current, *goal)
                frontier.put((priority, next_state))

    return cost_so_far[current] if current == goal else None


cost(grid)

286

In [6]:
for x, y in coords[n:]:
    grid[x, y] = 1
    if cost(grid) is None:
        print(f"{y},{x}")
        break

20,64
